Obtain cluster url for spark

In [ ]:
! kubectl cluster-info | grep master

Once cluster is accessible the above cells need not be re-run

In [ ]:
import pyspark
import os
import socket
import random

In [ ]:
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3.6'

In [ ]:
conf = pyspark.conf.SparkConf()

In [ ]:
k8sMaster = "https://10.0.0.1:443"

In [ ]:
conf = conf.setMaster("k8s://" + k8sMaster) \
    .setAppName("pyspark-notebook") \
    .set("spark.kubernetes.executor.container.image", "docker.io/guislander/spark-py:latest")  \
    .set("spark.kubernetes.namespace", "jhub") \
    .set("spark.kubernetes.authenticate.driver.serviceAccountName","spark")  \
    .set("spark.executor.instances", "2") \
    .set("spark.kubernetes.executor.request.cores","100m") \
    .set("spark.kubernetes.executor.limit.cores","1") \
    .set("spark.kubernetes.pyspark.pythonVersion","3") \
    .set("spark.driver.host", socket.gethostbyaddr(socket.gethostname())[2][0]) \
    .set("spark.driver.port", "29413") \
    .set("spark.driver.pod.name", socket.gethostname())

In [ ]:
sc = pyspark.context.SparkContext(conf=conf)

hello world

In [ ]:
def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1

NUM_SAMPLES = 1000
count = sc.parallelize(range(0, NUM_SAMPLES)) \
             .filter(inside).count()
print ("Pi is roughly %f" % (4.0 * count / NUM_SAMPLES))

In [ ]:
big_list = range(10000)
rdd = sc.parallelize(big_list, 2)
odds = rdd.filter(lambda x: x % 2 != 0)
odds.take(5)

In [ ]:
sc.stop()